# -------- Application Layer communicates with the backend and frontend

In [7]:

import requests
import anvil.server
import numpy as np
import json
import anvil.media
import os
import shutil
import re
import numpy as np
from helperFunctions import  img_url_extractor, extract_infos, load_image, save_image, available_actors, best_known_for_genre, average_rating_of_each_actor


# ------- Fast API URL ----------------
# To run make sure server unvicorn is running
FASTAPI_URL_SWAGGER = "http://127.0.0.1:8000/docs" # Shows all Endpoints at once
FASTAPI_URL = "http://127.0.0.1:8000/" # root
FASTAPI_CELEBS_URL = "http://127.0.0.1:8000/most_popular_celebrities"
FASTAPI_MOVIES = "http://127.0.0.1:8000/movies"

"http://127.0.0.1:8000/actors/1" # <----- you can try out different numbers [1 - 50]
"http://127.0.0.1:8000/actor/1" # <----- you can try out different numbers [1 - 50]
"http://127.0.0.1:8000/award/1" # <----- you can try out different numbers [1 - 50]
"http://127.0.0.1:8000/movie/3" # <----- you can try out different numbers [1 - 50]


# ----- Anvil Section: Setting up User Interface communication -----
SERVER_Key = "server_ATDO2RYYLYOUFL7T4GLFNOPK-EXXUFTIZNGRXOSVW"

## --- User Interface: concection
anvil.server.connect(SERVER_Key)


# ------- Celabrities data
csv_file = "data/actors_ids_names_birthdates.csv"

### --------------GET Methods -----------------

In [8]:
@anvil.server.callable
def display_all_actors():
    """Loads celebrities from FastAPI endpoint by making a request.
        processes their images, and returns a list of actor data.
    
    Args:
        None:
    Returns: 
        list (List): List of all available actors
    """
    res = requests.get(FASTAPI_CELEBS_URL)
    if res.status_code != 200:
        print("Error: Status code retured Error")
        return None
    img_path = "data/images"   
    # Ensure the image folder exists
    if not os.path.exists(img_path):
        print(f"Path {img_path} does not exist.")
        return None
    best_known_for = ""
    lists_of_actors = []
    result = res.json()
    for actor in result:
        img_name = actor["name"].lower().replace(" ", "_")
        name = actor["name"]
        birthdate = actor["birthday"]
        best_known_for = best_known_for_genre(name)
        
        lists_of_actors.append({
            "name": name,
            "birthdate": birthdate,
            "image": anvil.BlobMedia("image/JPEG", open(f"data/images/{img_name}.jpg", "rb").read()),
            "actor_genre": best_known_for
        })
    return lists_of_actors

# Keeps the uplink running (if needed)
#anvil.server.wait_forever()

In [9]:
@anvil.server.callable
def all_time_movies(person: str):
    """This function returns all the movies an actor stared in 
    In addition it returns the average rating of each actor
    Parameter: 
        name (str): Full name of an actor
        
    Return: 
        movies (list): List of movies an actor stared in
        average (float): Average Rating number """
    list_of_movies = []
    """This function returns the list of all movies an actor stared in """
    img_folder = person.lower().replace(" ", "_")

    top_movies = []
    res = requests.get(FASTAPI_CELEBS_URL)
    if res.status_code != 200:
        print("Status Error: Connection not possible")
        return None
    actors = res.json()
    id = None
    for data in actors:
        if person.lower() == data["name"].lower():
            id = data["id"]
    if id is None:
        return [] # Return an empty list if actor cannot be found
    movies = requests.get(f"http://127.0.0.1:8000/movie/{id}")
    actor_movies = movies.json()
    i = 1
    for movie in actor_movies:
    
        title = movie["title"]
        img_name = re.sub(r'[\/:*?"<>|]', '_', title)
        img_name = img_name.replace(" ", "_")
        rating = movie["rating"]
        genre = movie["genre"]
        year = movie["year"]
        top_movies.append(
            {
            "movie_image": anvil.BlobMedia("image/JPEG", open(f"data/images/movie_images_{img_folder}/{img_name}.jpg", "rb").read()),
            "movie_title": title,
            "movie_rating" : rating,
            "movie_genre": genre,
            "movie_year": year               
            }
        )
    return top_movies

In [10]:
@anvil.server.callable
def actor_profile(name_input):
    """This Function returns the profile of a specific actor when full name is given
    It sends a request to the fastapi endpoints to retieve the data its needs from an actor
    
    Args:
        name (str): Full name of an actor
    Returns: 
        profile (list): List of an actor's achievments
    """
    list_of_profiles = []

    res = requests.get(FASTAPI_CELEBS_URL)
    if res.status_code != 200:
        print("Error: Connection not possible")
        return None
    actors = res.json()
    id = None
    for data in actors:
        if name_input.lower() == data["name"].lower():
            id = data["id"]
    if id is None:
        return [] # Return empty list for actor not found      
    pros_res = requests.get(f"http://127.0.0.1:8000/actors/{id}")
    awards_res = requests.get(f"http://127.0.0.1:8000/award/{id}")
    awards = awards_res.json()
    profile = pros_res.json() 
    
    img_name = profile["name"].lower().replace(" ", "_")
    actor_name = profile["name"]
    birthdate = profile["birthday"]
    biography = profile["biography"]
    nominations = awards["nominations"]
    award_won = awards["award_won"]

    list_of_profiles.append(
        {
            "actor_name_profile": actor_name,
            "birthdate_profile": birthdate, 
            "biography": biography, 
            "image_profile": anvil.BlobMedia("image/JPEG", open(f"data/images/{img_name}.jpg", "rb").read()),
            "nominations": nominations,
            "won": award_won,
            "average_rating" : round(average_rating_of_each_actor(actor_name), 2)
            
        }
    )    
    return list_of_profiles

In [11]:

@anvil.server.callable
def get_actors_movies(person):
    """This function returns the list of all movies an actor stared in """
    img_folder = person.lower().replace(" ", "_")

    top_5_movies = []
    res = requests.get(FASTAPI_CELEBS_URL)
    if res.status_code != 200:
        print("Error: Connection not possible")
        return None
    actors = res.json()
    id = None
    for data in actors:
        if person.lower() == data["name"].lower():
            id = data["id"]
    if id is None:
        return [] # Return empty list if actor does not exist
    movies = requests.get(f"http://127.0.0.1:8000/movie/{id}")
    actor_movies = movies.json()
    i = 1
    for movie in actor_movies:
        if i <= 5:
            title = movie["title"]
            img_name = re.sub(r'[\/:*?"<>|]', '_', title)
            img_name = img_name.replace(" ", "_")
            rating = movie["rating"]
            genre = movie["genre"]
            year = movie["year"]
            top_5_movies.append(
                {
                "movie_image": anvil.BlobMedia("image/JPEG", open(f"data/images/movie_images_{img_folder}/{img_name}.jpg", "rb").read()),
                "movie_title": title,
                "movie_rating" : rating,
                "movie_genre": genre,
                "movie_year": year               
                }
            )
            i += 1
    return top_5_movies

In [12]:
@anvil.server.callable
def get_actor_awards(name):
    """This function returns a list of all awards an actor won"""
    res = requests.get("http://127.0.0.1:8000/most_popular_celebrities")
    if res.status_code != 200:
        print("Error: Connection not possible")
        return None
    
    actors = res.json()
    id = None
    for data in actors:
        if data["name"].lower() == name.lower():
            id = data["id"]
    if id is None: 
        return [] # Return empty list when actor cannot be found
    res_award = requests.get(f"http://127.0.0.1:8000/award/{id}") 
    awards_data = res_award.json()
    list_of_awards = []
    awards = awards_data["award_name"]
    years = awards_data["year"]
    
    for award, year in zip([awards[i] for i in range(len(awards))], [years[k] for k in range(len(years))]):
        list_of_awards.append(
            {
                "award_name": award, 
                "award_year": year
            }
        )
    return list_of_awards